In [1]:
import numpy as np
import scipy as sp
import time
import sys
sys.path.append('/usr/local/lib/python2.7/dist-packages/gmr-1.1_git-py2.7.egg')
import gmr as gaussian_regression

In [2]:
#Environment definition
from scipy import spatial
from explauto.utils import rand_bounds
from explauto.environment.environment import Environment
from explauto import SensorimotorModel
from explauto.sensorimotor_model import sensorimotor_models

In [3]:
from explauto.models.gmrinf import GMR

In [4]:
environment = Environment.from_configuration('simple_arm', 'mid_dimensional')

In [5]:
sm_cls, sm_configs = sensorimotor_models['nearest_neighbor']
model = sm_cls(environment.conf, **sm_configs['default'])

In [6]:
#Creation of the total dataset
def generateFwdDataset(_environment, _model, _n = 1000):
    random_motors = _environment.random_motors(n=_n)
   
    for m in random_motors:
        s = _environment.compute_sensori_effect(m)
        _model.update(m, s)
generateFwdDataset(environment, model)

In [7]:
x_g = environment.random_motors(n=1)
#Creation fof a local dataset depending on x_g
l_dist, l_index = model.model.imodel.fmodel.dataset.nn_x(x_g[0], 100)
l_nn = []
col1 = []
col2 = []
for index in l_index:
    col1.append(model.model.imodel.fmodel.dataset.get_xy(index)[0])
    col2.append(model.model.imodel.fmodel.dataset.get_xy(index)[1])
    l_nn.append(model.model.imodel.fmodel.dataset.get_xy(index))
lnn_nparray = np.array(l_nn)

In [7]:
data_list = model.model.imodel.fmodel.dataset.data

In [8]:
data_np = np.array(data_list).T

In [9]:
data = []
for i in range(len(data_np)):
    data.append(np.hstack(data_np[i,:]))

In [10]:
gmr = GMR(n_components=3)
gmr = gmr.from_samples(np.array(data))

In [11]:
conditional = gmr.condition([0, 1, 2, 3, 4, 5, 6], [-0.76151596,  0.85728845,  0.0987334 ,  0.46247676, -0.16366163,
         0.27212757,  0.08281898])

In [12]:
print conditional.priors

[ 0.00477281  0.98996846  0.00525874]


In [13]:
prediction = gmr.predict(np.array([0, 1, 2, 3, 4, 5, 6]), np.transpose(np.array([[-0.76151596],  [0.85728845],  [0.0987334],  [0.46247676], [-0.16366163],
         [0.27212757],  [0.08281898]])))


In [14]:
print prediction[0]

[ 0.8076785  -0.07856919]


In [15]:
proba = gmr.probability(np.transpose(np.array([[-0.76151596],  [0.85728845],  [0.0987334],  [0.46247676], [-0.16366163],
         [0.27212757],  [0.08281898], [0.84411128], [-0.07436732]])))
print proba

0.541884810781


In [16]:
reg = gmr.regression(np.array([0, 1, 2, 3, 4, 5, 6]), np.array([7,8]), np.transpose(np.array([[-0.76151596],  [0.85728845],  [0.0987334],  [0.46247676], [-0.16366163],
         [0.27212757],  [0.08281898]])), "lse")

In [17]:
print reg

[ 0.8076785  -0.07856919]


In [18]:
slse = gmr.regression(np.array([0, 1, 2, 3, 4, 5, 6]), np.array([7,8]), np.transpose(np.array([[-0.76151596],  [0.85728845],  [0.0987334],  [0.46247676], [-0.16366163],
         [0.27212757],  [0.08281898]])), "slse")

In [19]:
print slse

[ 0.8044823  -0.07751704]
